### Import Libraries

In [3]:
import requests
import calendar
import xml.etree.ElementTree as et
import pandas as pd

### Assign Log-in Information

In [4]:
api_key = 'c2303feba65bd969a88ec1533cb977'

user = 'barbieri.greg@bls.gov'
passwd = 'Rivert56%'

user = '125391'
access = '2da28127c62e7a5c718da9fd55b647a91a5315c4'

### Build Query

In [5]:
output = 'xml'

headers = {'Accept': 'application/{}'.format(output),
           'Content-Type': 'application/{}'.format(output)}

call_url = 'http://slimtimer.com/users/{}/time_entries'.format(user)

### Request Data

In [6]:
params = {'api_key': api_key,
          'access_token': access,
          'offset': '0',
          'range_start': '2019-05-10 00:00:00',
          'range_end': '2019-05-22 23:59:59'}

request = requests.get(call_url, params=params, headers=headers)

### Parse XML

In [7]:
xml = et.fromstring(request.text)

In [8]:
lst_dct = []

for table in xml.iter('time-entry'):
    dct = {}
    
    for child in table:
        if child.tag == 'task':
            for gchild in child:
                if gchild.tag == 'name':
                    dct[gchild.tag] = gchild.text
        else:
            dct[child.tag] = child.text
    
    lst_dct.append(dct)

### Dictionary to DataFrame

In [78]:
time_entries = pd.DataFrame(lst_dct)

In [79]:
time_entries['duration-in-seconds'] = time_entries['duration-in-seconds'].astype(int) / (60*60)
time_entries[['category','task']] = time_entries['name'].str.split("-", n=1, expand=True)

In [80]:
df_task = time_entries.groupby(by=['name'], as_index=False)['duration-in-seconds'].sum()
df['total_time'] = time_entries['duration-in-seconds'].sum()
df['task_proportion'] = df['duration-in-seconds'] / df['total_time']
df.rename(columns={'duration-in-seconds' : 'task_time'}, inplace=True)

In [84]:
df_cat = time_entries.groupby(by=['category'], as_index=False)['duration-in-seconds'].sum()
df_cat['total_time'] = time_entries['duration-in-seconds'].sum()
df_cat['category_proportion'] = df_cat['duration-in-seconds'] / df_cat['total_time']
df_cat.rename(columns={'duration-in-seconds' : 'cat_time'}, inplace=True)

### Summarize

In [83]:
df.sort_values(by=['category_proportion'], ascending=False)

,name,task_time,total_time,task_proportion
12,SMS Dev - General,18.447500,63.612778,0.289997
7,Misc - Lunch,8.973889,63.612778,0.141071
18,Training - Python,5.941667,63.612778,0.093404
0,Admin - Meeting,4.330556,63.612778,0.068077
6,"Misc - Breaks (Restroom, Chatting, Etc.)",4.100556,63.612778,0.064461
13,SMS Dev - PO Meeting,3.515556,63.612778,0.055265
15,Seminar - Emotional Intelligence,2.995278,63.612778,0.047086
8,Misc - Misc/Admin,2.493056,63.612778,0.039191
14,SMS Dev - Story Writing,2.429167,63.612778,0.038187
2,Admin - OPLC Town Hall,1.998889,63.612778,0.031423


In [86]:
df_cat.sort_values(by=['category_proportion'], ascending=False)

,category,cat_time,total_time,task_proportion
4,SMS Dev,24.392222,63.612778,0.383448
1,Misc,15.902778,63.612778,0.249993
0,Admin,9.886111,63.612778,0.155411
7,Training,5.941667,63.612778,0.093404
5,Seminar,2.995278,63.612778,0.047086
6,TPOPS,2.223333,63.612778,0.034951
2,OCET,1.063056,63.612778,0.016711
3,SMS,0.830000,63.612778,0.013048
8,Vacation,0.378333,63.612778,0.005947
